# 13. Parallelizing Neural Netowrk Training with TensorFlow

In [1]:
%run -i 'watermark.py'

2020-07-31 21:44:21
----------------------
python		3.6.7
----------------------
numpy		1.18.5
scipy		1.4.1
pandas		0.25.1
matplotlib	3.1.1
imageio		2.5.0
----------------------
ipython		7.8.0
----------------------
sklearn		0.20.4
tensorflow	2.3.0
nltk		3.2.4
----------------------
networkx	2.0


## 13.01. TensorFlow and training performance

### 13.01.01. Performance challenges

### 13.01.02. What is TensorFlow?

![Fig.13.1](https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch13/images/01.png)

### 13.01.03. How we will learn TensorFlow

## 13.02. First steps with TensorFlow

### 13.02.01. Installing TensorFlow

### 13.02.02. Creating ttensors in TensorFlow



In [2]:
#import tensorflow as tf
#import numpy as np

np.set_printoptions(precision=3)
a = np.array([1, 2, 3], dtype=np.int32)
b = [4, 5, 6]
t_a = tf.convert_to_tensor(a)
t_b = tf.convert_to_tensor(b)
print(t_a)
print(t_b)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([4 5 6], shape=(3,), dtype=int32)


In [3]:
t_ones = tf.ones((2, 3))
t_ones.shape

TensorShape([2, 3])

In [4]:
t_ones.numpy()

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [5]:
const_tensor = tf.constant([1.2, 5, np.pi], dtype=tf.float32)
print(const_tensor)

tf.Tensor([1.2   5.    3.142], shape=(3,), dtype=float32)


## 13.02.03. Manipulating the data type and shape of a tensor

`tf.cast()`

In [6]:
t_a_new = tf.cast(t_a, tf.int64)
print(t_a_new.dtype)

<dtype: 'int64'>


`tf.transpose()`, `tf.reshape()`, `tf.squeeze()`

In [7]:
t = tf.random.uniform(shape=(3, 5))
t_tr = tf.transpose(t)
print(t.shape, '-->', t_tr.shape)

(3, 5) --> (5, 3)


In [8]:
t = tf.zeros((30,))
t_reshape = tf.reshape(t, shape=(5, 6))
print(t_reshape.shape)

(5, 6)


In [9]:
t = tf.zeros((1, 2, 1, 4, 1))
t_sqz = tf.squeeze(t, axis=(2,4))
print(t.shape, '-->', t_sqz.shape)

(1, 2, 1, 4, 1) --> (1, 2, 4)


### 13.02.04. Applying matheatical operations to tensors 

In [10]:
# tf.random.set_seed(1)
t1 = tf.random.uniform(shape=(5,2), minval=1.0, maxval=1.0)
t2 = tf.random.normal(shape=(5,2), mean=0.0, stddev=1.0)

In [11]:
t3 = tf.multiply(t1, t2).numpy()
print(t3)

[[-0.153  0.77 ]
 [-1.878  0.25 ]
 [-0.173  0.094]
 [ 0.024 -0.177]
 [ 0.159  0.219]]


`tf.math.reduce_mean()`, `tf.math.reduce_sum()`, `tf.math.reduce_std()`

In [12]:
t4 = tf.math.reduce_mean(t1, axis=0)
print(t4)

tf.Tensor([1. 1.], shape=(2,), dtype=float32)


$ t_1 \times t_2^T $

`tf.linalg.matmul()`

In [13]:
t5 = tf.linalg.matmul(t1, t2, transpose_b=True)
print(t5.numpy())

[[ 0.616 -1.629 -0.079 -0.153  0.378]
 [ 0.616 -1.629 -0.079 -0.153  0.378]
 [ 0.616 -1.629 -0.079 -0.153  0.378]
 [ 0.616 -1.629 -0.079 -0.153  0.378]
 [ 0.616 -1.629 -0.079 -0.153  0.378]]


In [14]:
t6 = tf.linalg.matmul(t1, t2, transpose_a=True)
print(t6.numpy())

[[-2.021  1.156]
 [-2.021  1.156]]


`tf.norm()`
$L^p$ e.g., $L^2$

In [15]:
norm_t1 = tf.norm(t1, ord=2, axis=1).numpy()
print(norm_t1)

[1.414 1.414 1.414 1.414 1.414]


### 13.02.05. Split, stack, and concatenate tensors

`tf.split()`

providing the num of splits

In [17]:
tf.random.set_seed(1)
t = tf.random.uniform((6,))
print(t.numpy())

t_spllits = tf.split(t, num_or_size_splits=3)
[item.numpy() for item in t_spllits]

[0.165 0.901 0.631 0.435 0.292 0.643]


[array([0.165, 0.901], dtype=float32),
 array([0.631, 0.435], dtype=float32),
 array([0.292, 0.643], dtype=float32)]

providing the sizes of different splits


In [18]:
tf.random.set_seed(1)
t = tf.random.uniform((5,))
print(t.numpy())

t_splits = tf.split(t, num_or_size_splits=[3, 2])
[item.numpy() for item in t_splits]

[0.165 0.901 0.631 0.435 0.292]


[array([0.165, 0.901, 0.631], dtype=float32),
 array([0.435, 0.292], dtype=float32)]

`tf.stack()`, `tf.concat()`

In [19]:
A = tf.ones((3,))
B = tf.zeros((2,))
C = tf.concat([A, B], axis=0)
print(C.numpy())

[1. 1. 1. 0. 0.]


In [20]:
A = tf.ones((3,))
B = tf.zeros((3,))
S = tf.stack([A, B], axis=1)
print(S)

tf.Tensor(
[[1. 0.]
 [1. 0.]
 [1. 0.]], shape=(3, 2), dtype=float32)


[TensorFlow docs](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf)

## 13.03. Builing input pipleines using tf.data - the TensorFlow Dateaset API

### 13.03.01. Creating a TensorFlow Dataset from existing tensors

In [22]:
a = [1.2, 3.4, 7.5, 4.1, 5.0, 1.0]
ds = tf.data.Dataset.from_tensor_slices(a)
print(ds)

<TensorSliceDataset shapes: (), types: tf.float32>


In [24]:
for item in ds:
    print(item)

tf.Tensor(1.2, shape=(), dtype=float32)
tf.Tensor(3.4, shape=(), dtype=float32)
tf.Tensor(7.5, shape=(), dtype=float32)
tf.Tensor(4.1, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


batch 

In [25]:
ds_batch = ds.batch(3)
for i, elem in enumerate(ds_batch, 1):
    print('batch {}:'.format(i), elem.numpy())

batch 1: [1.2 3.4 7.5]
batch 2: [4.1 5.  1. ]


`.batch()` method has optional arg `drop_remainder`

### 13.03.02. Combining two tensors into a joint dataset

In [27]:
tf.random.set_seed(1)
t_x = tf.random.uniform([4,3], dtype=tf.float32)
t_y = tf.range(4)

In [28]:
ds_x = tf.data.Dataset.from_tensor_slices(t_x)
ds_y = tf.data.Dataset.from_tensor_slices(t_y)

ds_joint = tf.data.Dataset.zip((ds_x, ds_y))

for example in ds_joint:
    print('  x:', example[0].numpy(), 
          '  y:', example[1].numpy())

  x: [0.165 0.901 0.631]   y: 0
  x: [0.435 0.292 0.643]   y: 1
  x: [0.976 0.435 0.66 ]   y: 2
  x: [0.605 0.637 0.614]   y: 3
